In [1]:
"""
3 ways to do SentenceLDA:
-> Do Vanilla-LDA: Find weighting for documents. Find word->topic, average over sentences
-> Do Vanilla-LDA where document = sentence, average across sentence for the document
-> ??
"""

'\n3 ways to do SentenceLDA:\n-> Do Vanilla-LDA: Find weighting for documents. Find word->topic, average over sentences\n-> Do Vanilla-LDA where document = sentence, average across sentence for the document\n-> ??\n'

In [1]:
import pickle
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

from gensim.models import Phrases
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_numeric, strip_punctuation, strip_short, stem_text
import matplotlib.pyplot as plt

from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary

In [2]:
path = "../../Files/"
file = "processed_data_withpunc.csv"
data = pd.read_csv(path + file)

In [3]:
data.head()

,Unnamed: 0,cik,ticker_x,filing_date,item1a_risk,item7_mda,year_x,filing_year_x,perm_id,ticker_y,year_y,company_name,is_dividend_payer,dps_change,is_dps_cut,z_environmental,d_environmental,sector,filing_year_y
0,0,1750,AIR,2010-07-16,item risk factor the follow descript princip r...,item management' discuss and analysi financi c...,2010,2010,4332,AIR,2010,AAR CORP,0.0,2.000000,NaN,NaN,NaN,Industrials,2010
1,1,2034,ACET,2010-09-10,>item risk factor you carefulli consid follow ...,>item manag discuss analysi financi condit res...,2010,2010,2883,ACET,2010,ACETO CORP,1.0,0.000000,0.0,NaN,NaN,Industrials,2010
2,2,2491,BYI,2010-08-26,item risk factor our busi invest common stock ...,item management' discuss and analysi financi c...,2010,2010,2791,BYI,2010,BALLY TECHNOLOGIES INC,0.0,NaN,NaN,NaN,NaN,Tech,2010
3,3,2969,APD,2010-11-23,>item risk factor you read follow risk factor ...,>item manag discuss and analysi financi condit...,2010,2010,2799,APD,2010,AIR PRODUCTS & CHEMICALS INC,1.0,0.149398,0.0,0.620576,-1.857054,Commodities,2010
4,4,3545,ALCO,2010-12-14,>item risk factors. alico oper involv vari deg...,>item manag discuss analysi financi condit res...,2010,2010,1824,ALCO,2010,ALICO INC,1.0,0.181818,0.0,NaN,NaN,Consumer,2010


In [4]:
SELECTED_YEARS = set(list(range(2009,2012)))
display(SELECTED_YEARS)
filtered_by_year = data[data.year_x.isin(SELECTED_YEARS)]
items = {
    'item1a': filtered_by_year['item1a_risk'],
    'item7': filtered_by_year['item7_mda']
}

item_sentences = {}

{2009, 2010, 2011}

In [5]:
# No bigrams, or lemmanzizing
tokenizer = RegexpTokenizer(r'\w+')
for item_key in items.keys():
    sentences = []
    for idx in range(len(items[item_key])):
        full_string = re.sub("[\(;,\)]", "", items[item_key].iloc[idx].lower())
        these_sentences = full_string.split(".")
        these_sentences = [[word for word in tokenizer.tokenize(sent) if (len(word) > 2 and not word.isnumeric())] for sent in these_sentences]
        items[item_key].iloc[idx] = these_sentences
        
        sentences.extend(these_sentences)
    
    item_sentences[item_key] = sentences

/home/harshul/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/harshul/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.array(value)
<ipython-input-5-03804822a0b6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [6]:
display(len(item_sentences["item1a"]))
display(item_sentences["item1a"][0])
display(len(item_sentences["item1a"][1]))

746720

['item',
 'risk',
 'factor',
 'the',
 'follow',
 'descript',
 'princip',
 'risk',
 'inher',
 'business']

7

In [7]:
dictionaries = {}
for item in item_sentences:
    dictionaries[item] = Dictionary(item_sentences[item])

In [8]:
corpus = {}
for item in item_sentences:
    corpus[item] = [dictionaries[item].doc2bow(doc) for doc in item_sentences[item]]

In [9]:
for item in item_sentences:
    print(item + ':')
    print('\tNumber of unique tokens: %d' % len(dictionaries[item]))
    print('\tNumber of documents: %d' % len(corpus[item]))

item1a:
	Number of unique tokens: 39437
	Number of documents: 746720
item7:
	Number of unique tokens: 52347
	Number of documents: 1161497


In [10]:
id2word = {}
for item in items:
    temp = dictionaries[item][0] # Initialize id2token mappings
    id2word[item] = dictionaries[item].id2token

In [11]:
import pickle

base_name = "2009-2011_{}_{}.pkl"
str_mapping = {
    'corpus': corpus,
    'id2word': id2word
}

for item in items:
    for obj in str_mapping:
        display(item + "_" + obj)
        with open(path+base_name.format(item, obj), 'wb') as file:
            pickle.dump(str_mapping[obj][item], file)

'item1a_corpus'

'item1a_id2word'

'item7_corpus'

'item7_id2word'

In [28]:
str_mapping["id2word"]["item1a"]

{0: 'business',
 1: 'descript',
 2: 'factor',
 3: 'follow',
 4: 'inher',
 5: 'item',
 6: 'princip',
 7: 'risk',
 8: 'the',
 9: 'advers',
 10: 'affect',
 11: 'aviat',
 12: 'commerci',
 13: 'industry',
 14: 'condit',
 15: 'econom',
 16: 'greatli',
 17: 'overal',
 18: 'product',
 19: 'provid',
 20: 'servic',
 21: 'acut',
 22: 'air',
 23: 'attack',
 24: 'cyclic',
 25: 'decad',
 26: 'declin',
 27: 'high',
 28: 'histor',
 29: 'industri',
 30: 'neg',
 31: 'oil',
 32: 'outbreak',
 33: 'past',
 34: 'prices',
 35: 'respiratori',
 36: 'sars',
 37: 'septemb',
 38: 'sever',
 39: 'subsequ',
 40: 'syndrome',
 41: 'terrorism',
 42: 'terrorist',
 43: 'travel',
 44: 'war',
 45: 'bankruptci',
 46: 'certain',
 47: 'custom',
 48: 'events',
 49: 'file',
 50: 'forc',
 51: 'future',
 52: 'protect',
 53: 'result',
 54: 'years',
 55: 'abroad',
 56: 'chang',
 57: 'decreas',
 58: 'demand',
 59: 'deterior',
 60: 'more',
 61: 'recently',
 62: 'state',
 63: 'transport',
 64: 'unit',
 65: 'economies',
 66: 'global',


In [16]:
params = {
    'num_topics': 30,
    'chunksize': 2000,
    'passes': 20,
    'iterations': 400,
    'eval_every': None,
    'alpha': 'asymmetric',
    'eta': 'auto'
}

In [18]:
models = {}
for item in items:
    display(item)
    models[item] = LdaMulticore(
        corpus=corpus[item],
        id2word=id2word[item],
        workers=32,
        **params
    )

'item1a'

OSError: [Errno 12] Cannot allocate memory

In [14]:
del data

In [13]:
with open(path+"items", 'wb') as file:
    pickle.dump(items, file)